<a href="https://colab.research.google.com/github/Swayamprakashpatel/DD/blob/main/FIngerprint_SMILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [2]:
#@title DOWNLOAD DATA FROM KAGGLE
# DOWNLOAD DATA FROM KAGGLE (!IMPORTANT!: REFRESH RUNTIME BEFORE RE-RUNNING THE CODE)
#%%capture
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

#Make Directory of Kaggle and set its permission for access.
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

# Download Data from Kaggle Fast and Unzip them in /content
!kaggle datasets download -d drswayamprakashpatel/DD-Dataset-csv  -p /content # For model download

#Unzip data (Two Folders - Training and Validation)
import os
os.chdir('/content')
#create a directory named train/
!unzip -q DD-Dataset-csv.zip #Unzip Model

Saving kaggle.json to kaggle.json
kaggle.json
 18% 1.00M/5.59M [00:00<00:00, 5.73MB/s]
100% 5.59M/5.59M [00:00<00:00, 21.0MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Load the dataset
data = pd.read_csv('/content/DATASET_Final.csv')  # Replace with the path to your dataset
#data = data.iloc[0:10000,:]
# Split the data into input and output columns
X = data.iloc[:, 7:888]  # Input columns
y = data['SMILE']      # Output column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshape the input data
X_train_reshaped = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshaped = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Preprocess the output data (SMILES)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(y_train)
y_train_sequences = tokenizer.texts_to_sequences(y_train)
y_test_sequences = tokenizer.texts_to_sequences(y_test)

# Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in y_train_sequences + y_test_sequences)

# Pad sequences to ensure consistent length
y_train_padded = pad_sequences(y_train_sequences, maxlen=max_sequence_length)
y_test_padded = pad_sequences(y_test_sequences, maxlen=max_sequence_length)

# Define the model
input_shape = (1, X_train.shape[1])
output_vocab_size = len(tokenizer.word_index) + 1
hidden_units = 28

inputs = Input(shape=input_shape)

##ENCODER###
encoder = LSTM(hidden_units, return_sequences=True)(inputs) #return_sequences=True in the first LSTM layer (encoder). This will ensure that the output of the first LSTM layer is a sequence, which can then be passed as input to the second LSTM layer.
encoder_1 = LSTM(hidden_units, return_sequences=False)(encoder) # Extra LSTM Layer: 
hidden_layer = Dense(hidden_units, activation='relu')(encoder_1)# Hidden layer in encoder

##DECODER##
decoder = Dense(hidden_units, activation='relu')(hidden_layer)
hidden_layer_1 = Dense(hidden_units, activation='relu')(decoder) #Hidden laye rin decoder
outputs = Dense(max_sequence_length, activation='softmax')(hidden_layer_1)  # Updated output layer units


model = Model(inputs=inputs, outputs=outputs)

# Compile and train the model
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(X_train_reshaped, y_train_padded, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test_padded))



import tensorflow as tf
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

filepath = '/content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5'

monitor = 'val_accuracy'
mode = 'max'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor = monitor, mode = mode, save_best_only=True, Save_weights_only = False, verbose = 1), 
              tf.keras.callbacks.EarlyStopping(monitor= monitor, mode = mode, patience=250, verbose =1), [tensorboard_callback]]


model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.8), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
hist = model.fit(X_train_reshaped, y_train_padded, epochs= 2000, callbacks=[checkpoint],validation_data=(X_test_reshaped, y_test_padded), batch_size= None)

Epoch 1/2000
683/684 [============================>.] - ETA: 0s - loss: 10152764456065660638749261824.0000 - accuracy: 0.0612
Epoch 1: val_accuracy improved from -inf to 0.06126, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 16s 15ms/step - loss: 10153450379797297454716551168.0000 - accuracy: 0.0612 - val_loss: 10437147726847312108340641792.0000 - val_accuracy: 0.0613
Epoch 2/2000
683/684 [============================>.] - ETA: 0s - loss: 11423791756113267082838147072.0000 - accuracy: 0.0633
Epoch 2: val_accuracy did not improve from 0.06126
684/684 [==============================] - 8s 11ms/step - loss: 11422305391262783862007136256.0000 - accuracy: 0.0633 - val_loss: 10479922922449145354686300160.0000 - val_accuracy: 0.0613
Epoch 3/2000
680/684 [============================>.] - ETA: 0s - loss: 11457899048035793095394066432.0000 - accuracy: 0.0634
Epoch 3: val_accuracy did not improve from 0.06126
684/684 [===============

In [ ]:
print(y_train)

55    CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=CC(=C(C=C3)F)Cl)...
88        COC1=C(C=C2C(=C1)C(=NC=N2)OC3=CC(=CC=C3)Br)OC
26    CCN(CCOC1=CC=C(C=C1)NC2=NC=C3C=C(C(=O)N(C3=N2)...
42    CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...
69    C1=CC(=CC(=C1)Br)NC2=NC=NC3=C2C(=CC=C3)[N+](=O...
                            ...                        
60    CN1CCC(CC1)NC2=NC=C3C(=N2)C(=NC=N3)NC4=CC(=C(C...
71       C1=CC=C(C=C1)CCN2C(=CC3=CC=C(C=C3)O)C(=O)NC2=O
14               C1=CC=C2C(=C1)C(=NN2)C3=NC4=CC=CC=C4N3
92    COC1=C(C=C2C(=C1)C(=NC=N2)C3=CNC4=C3C=C(C=C4)B...
51    CC1=C2C(=NC=NN2C=C1NC(=O)OCC3COCCN3)NC4=CC5=C(...
Name: SMILE, Length: 80, dtype: object


In [ ]:


#FOR PREDICTION
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5')

# Prepare the input data
input_data = X_test.iloc[0].values  # Replace with your input data of length 881
input_reshaped = input_data.reshape(1, 1, len(input_data))

# Make predictions
predictions = model.predict(input_reshaped)

# Decode the predicted sequence
predicted_sequence = tokenizer.sequences_to_texts(predictions.argmax(axis=2))[0]

OSError: ignored